# FINE TUNING PRE-TRAINED MODELS

In [1]:
import numpy as np
import sklearn
import os
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

In [2]:
CONFIGURATION = {
    "CLASS_NAMES" : ['angry', 'happy', 'sad'],
    "BATCH_SIZE" : 32,
    "IMAGE_SIZE" : 256,
    "LEARNING_RATE" : 0.01,
    "N_EPOCHS" : 20,
    "DROPOUT_RATE": 0.0,
    "REGULARIZATION_RATE" : 0.0,
    "N_FILTERS" : 6,
    "KERNEL_SIZE" : 3,
    "N_STRIDES" : 1,
    "POOL_SIZE" : 2,
    "N_DENSE_1" : 1024,
    "N_DENSE_2" : 128,
    "NUM_CLASSES" : 3
}

trainDirectory = "/Users/aman/Documents/Work/Machine Learning/Computer-Vision-TensorFlow/Human-Emotions-Detection/Dataset/Emotions Dataset/Emotions Dataset/train"
testDirectory = "/Users/aman/Documents/Work/Machine Learning/Computer-Vision-TensorFlow/Human-Emotions-Detection/Dataset/Emotions Dataset/Emotions Dataset/test"

trainDataset = tf.keras.utils.image_dataset_from_directory(
    trainDirectory,
    labels='inferred',
    label_mode='categorical',
    class_names=CONFIGURATION["CLASS_NAMES"],
    color_mode='rgb',
    batch_size=CONFIGURATION["BATCH_SIZE"],
    image_size=(CONFIGURATION["IMAGE_SIZE"], CONFIGURATION["IMAGE_SIZE"]),
    shuffle=True,
    seed=99,
    validation_split=0.2,
    subset='training',
)

valDataset = tf.keras.utils.image_dataset_from_directory(
    trainDirectory,
    labels='inferred',
    label_mode='categorical',
    class_names=CONFIGURATION["CLASS_NAMES"],
    color_mode='rgb',
    batch_size=CONFIGURATION["BATCH_SIZE"],
    image_size=(CONFIGURATION["IMAGE_SIZE"], CONFIGURATION["IMAGE_SIZE"]),
    shuffle=True,
    seed=99,
    validation_split=0.2,
    subset='validation',
)

testDataset = tf.keras.utils.image_dataset_from_directory(
    testDirectory,
    labels='inferred',
    label_mode='categorical',
    class_names=CONFIGURATION["CLASS_NAMES"],
    color_mode='rgb',
    batch_size=CONFIGURATION["BATCH_SIZE"],
    image_size=(CONFIGURATION["IMAGE_SIZE"], CONFIGURATION["IMAGE_SIZE"]),
    shuffle=True,
    seed=99,
    validation_split=None,
    subset=None,
)

trainDataset = trainDataset.prefetch(tf.data.AUTOTUNE)
testDataset = testDataset.prefetch(tf.data.AUTOTUNE)
valDataset = valDataset.prefetch(tf.data.AUTOTUNE)

Found 6799 files belonging to 3 classes.
Using 5440 files for training.
Found 6799 files belonging to 3 classes.
Using 1359 files for validation.
Found 2278 files belonging to 3 classes.


2025-01-30 15:04:44.145343: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2025-01-30 15:04:44.145367: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 18.00 GB
2025-01-30 15:04:44.145375: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 6.00 GB
2025-01-30 15:04:44.145388: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-01-30 15:04:44.145398: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


## Loading The Pre-Trained Model

In [16]:
# Loading Pre-Trained Efficient Net B4 Model trained on ImageNet Dataset
backbone = tf.keras.applications.EfficientNetB4(
    include_top = False, # Include Classifier or Not
    weights = 'imagenet',
    input_shape = (CONFIGURATION["IMAGE_SIZE"], CONFIGURATION["IMAGE_SIZE"], 3)
)

## Defining Classification Model Using Pre-Trained Model as Feature Extractor Using Functional API

## Training Without Updating Pre-Trained Paramaters

In [19]:
backbone.trainable = False

In [20]:
input = tf.keras.layers.Input((CONFIGURATION["IMAGE_SIZE"], CONFIGURATION["IMAGE_SIZE"], 3))

x = backbone(input, training = False)

x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(CONFIGURATION['N_DENSE_1'], activation='relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dense(CONFIGURATION['N_DENSE_2'], activation='relu')(x)
output = tf.keras.layers.Dense(CONFIGURATION['NUM_CLASSES'], activation='softmax')(x)
 
fineTunedModel = tf.keras.models.Model(input, output)

fineTunedModel.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_15 (InputLayer)     │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb4 (Functional)     │ (None, 8, 8, 1792)     │    17,673,823 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_5      │ (None, 1792)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 1024)           │     1,836,032 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 1024)           │         4,096 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 128)            │       131,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,645,538 (74.94 MB)

 Trainable params: 1,969,667 (7.51 MB)

 Non-trainable params: 17,675,871 (67.43 MB)

In [21]:
lossFunction = tf.keras.losses.CategoricalCrossentropy()
METRICS = [tf.keras.metrics.CategoricalAccuracy(name="accuracy"), tf.keras.metrics.TopKCategoricalAccuracy(k=2, name="top_k_accuracy")]

In [22]:
fineTunedModel.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=CONFIGURATION["LEARNING_RATE"]),
    loss = lossFunction,
    metrics=METRICS
)

In [23]:
history = fineTunedModel.fit(
    trainDataset.take(10), # Training on a small dataset
    validation_data = valDataset,
    epochs = CONFIGURATION['N_EPOCHS'],
    verbose=1,
)

Epoch 1/20


2025-01-30 15:24:06.470031: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


10/10 ━━━━━━━━━━━━━━━━━━━━ 33s 2s/step - accuracy: 0.4915 - loss: 6.2065 - top_k_accuracy: 0.8218 - val_accuracy: 0.5040 - val_loss: 21.7150 - val_top_k_accuracy: 0.7557
Epoch 2/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step - accuracy: 0.5912 - loss: 5.3536 - top_k_accuracy: 0.8587 - val_accuracy: 0.4650 - val_loss: 22.1155 - val_top_k_accuracy: 0.8190
Epoch 3/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step - accuracy: 0.6557 - loss: 1.6768 - top_k_accuracy: 0.9340 - val_accuracy: 0.3319 - val_loss: 16.8002 - val_top_k_accuracy: 0.7469
Epoch 4/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step - accuracy: 0.6557 - loss: 2.4237 - top_k_accuracy: 0.8773 - val_accuracy: 0.3650 - val_loss: 9.4549 - val_top_k_accuracy: 0.6519
Epoch 5/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step - accuracy: 0.6789 - loss: 1.6233 - top_k_accuracy: 0.9064 - val_accuracy: 0.5011 - val_loss: 10.6160 - val_top_k_accuracy: 0.8124
Epoch 6/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step - accuracy: 0.7248 - loss: 1.4812 - top_k_accuracy: 0.907

## Training The Model Again But Now With Trainable for Backbone set to True & a very Small Learning Rate

In [26]:
backbone.trainable = True

In [27]:
fineTunedModel.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=CONFIGURATION["LEARNING_RATE"]/100),
    loss = lossFunction,
    metrics=METRICS
)

In [28]:
history = fineTunedModel.fit(
    trainDataset.take(10), # Training on a small dataset
    validation_data = valDataset,
    epochs = CONFIGURATION['N_EPOCHS'],
    verbose=1,
)

Epoch 1/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 128s 6s/step - accuracy: 0.5072 - loss: 21.3122 - top_k_accuracy: 0.7764 - val_accuracy: 0.5077 - val_loss: 12.8549 - val_top_k_accuracy: 0.7999
Epoch 2/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 43s 4s/step - accuracy: 0.5529 - loss: 11.0458 - top_k_accuracy: 0.8274 - val_accuracy: 0.4915 - val_loss: 11.7590 - val_top_k_accuracy: 0.8197
Epoch 3/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 43s 4s/step - accuracy: 0.5568 - loss: 6.8853 - top_k_accuracy: 0.8632 - val_accuracy: 0.4893 - val_loss: 11.3573 - val_top_k_accuracy: 0.8300
Epoch 4/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 44s 4s/step - accuracy: 0.6764 - loss: 3.8358 - top_k_accuracy: 0.9324 - val_accuracy: 0.4820 - val_loss: 11.5820 - val_top_k_accuracy: 0.8227
Epoch 5/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 45s 5s/step - accuracy: 0.7610 - loss: 3.1444 - top_k_accuracy: 0.9540 - val_accuracy: 0.4687 - val_loss: 11.7297 - val_top_k_accuracy: 0.8153
Epoch 6/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 43s 4s/step - accuracy: 0.7861 - loss: 2.3075 - top_k_